<a href="https://colab.research.google.com/github/bees4ever/ai-workshop-2022/blob/main/YOLO_Workshop_Capgemini_2022_Exercise_04_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Detection Workshop
*Capgemini Challenge*

Catch the thief through image analysis and prove your analytical skill!

Estimated Duration:

## Theft of a branded teddy bear

A valuable teddy bear was stolen from the shopping center. Witnesses saw the thief with the teddy bear in his arms. Your task is to search the surveillance camera images in an automated way to hand over a picture of the person to the police.
To do this, extract all the crops with a teddy bear. Find the thief by overlapping the bounding boxes of the teddy bear and the person.


Note: The data are downloaded from kaggle and underly different lizenses, even unkown.

- https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset (CC0 lizense)
- https://www.kaggle.com/datasets/fmena14/crowd-counting (unkown)



## Step 1 - Load Dataset


In [ ]:
!wget https://github.com/bees4ever/ai-workshop-2022/raw/main/surveillance_dataset/workshop_task_04_dataset.zip
!unzip workshop_task_04_dataset

As we learned from previous task, we

In [ ]:
# Load YOLO v5
!git clone  'https://github.com/ultralytics/yolov5.git'
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

In [ ]:
#' Run object detection
# !python yolov5/detect.py --source /content/workshop_task_04_dataset  --save-crop

In [ ]:
import torch
# Model
model = torch.hub.load('ultralytics/yolov3', 'yolov3')  # oder yolov3-spp, yolov3-tiny, custom# Inference für Bild erstellen



## Step 2 - Load prepared Functions

Run the following code snippet to load some prepared function which you should use for your calculation, later.


In [ ]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou


import numpy as np
def torch2dict(box):
  return {'x1': box[0].item(), 'x2': box[2].item(), 'y1': box[1].item(), 'y2': box[3].item()}

def getLabel(x):
  return " ".join(x['label'].split(" ")[0:-1])

def checkOverlap(overlap, label1, label2):
  return (getLabel(overlap["image1"]) == label1 and getLabel(overlap["image2"]) == label2) or (getLabel(overlap["image1"]) == label2 and getLabel(overlap["image2"]) == label1)

def box2Height(box):
  box_encoded = torch2dict(box)
  return box_encoded['y2'] - box_encoded['y1']

def boxSimilarHeight(box1, box2):
  heights = [box2Height(box1), box2Height(box2)]
  heights = np.sort(heights)
  return heights[0] / heights[1] >= 0.60
  



## Step 3 - Your turn, detect a person with a big teddy close to it

First, we provide some hints, then you also need to fill the prepared code snippet with your solutions.

If you want to write a complete own function you are more then invited to do so.


## Hints for your solution:

- use the function provided above, called `get_iou`
- as shown below, use `itertools.combinations` to compare all objects
- interesting objects regarding stolen teddy might be objects from almost same height, you can use the function `boxSimilarHeight`


In [ ]:
#' TODO: fill all <> with the missing solution!

import os
from progressbar import progressbar
import itertools
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from google.colab.patches import cv2_imshow

for img in progressbar(os.listdir("workshop_task_04_dataset")):
  img_path = os.path.join("workshop_task_04_dataset", img)
  results = model(img_path)
  crops = results.crop() 
  # xmin    ymin    xmax   ymax  confidence  class
  # bounding_boxes contains all found bounding boxes of current image
  bounding_boxes = results.xyxy[0]
  overlap_crops = []
  
  
  for (box1, crop1), (box2, crop2) in itertools.combinations(zip(bounding_boxes, crops), 2):
    overlap = get_iou(torch2dict(box1), torch2dict(box2))
    if overlap > 0 and overlap < 1:
      overlap_crops.append({"image1": crop1, "image2": crop2, "overlap": overlap, "box1": box1, "box2": box2})
  
  #' TODO: 
  for <> in overlap_crops:
      #' TODO, 
    if checkOverlap(overlap, <label1>, <label2>) and boxSimilarHeight(<>, <>):
      cv2_imshow(overlap["image1"]["im"])
      cv2_imshow(overlap["image2"]["im"])
      
      #' Hint: with getLabel(overlap["image1"]) you can print the label name
      #' TODO, what should be returned?
      print(<>)
      
      

